In [ ]:
import numpy as np
import pandas as pd
import fastf1
from src.plotset import setup_plot, save_fig

from fastf1 import plotting
import matplotlib.pyplot as plt
import seaborn as sns

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025,16,'R')
session.load()

In [ ]:
drivers = session.drivers
drivers = [session.get_driver(drv).Abbreviation for drv in drivers]

In [ ]:
temp = session.laps.pick_drivers('PIA').pick_quicklaps().copy()
temp['Stint_Compound'] = temp.apply(lambda x: str(int(x['Stint'])) + '_'+ x['Compound'],axis=1)

In [ ]:
comp_palette = {'1_SOFT': '#da291c',
                '2_SOFT': '#da291c',
                '3_SOFT': '#da291c',
                '1_MEDIUM': '#ffd12e',
                '2_MEDIUM': '#ffd12e',
                '3_MEDIUM': '#ffd12e',
                '1_HARD': '#f0f0ec',
                '2_HARD': '#f0f0ec',
                '3_HARD': '#f0f0ec'
}

In [ ]:
setup_plot(xyticksize=24,axeslabel=28, figtitle=40)

fig, ax = plt.subplots(figsize=(15,8))

for drv in drivers[:5]:

    df = session.laps.pick_drivers(drv).pick_quicklaps(1.05).copy()
    df['LapTime'] = df['LapTime'].dt.total_seconds()
    df['Stint_Compound'] = df.apply(lambda x: str(int(x['Stint'])) + '_'+ x['Compound'],axis=1)

    color = plotting.get_driver_color(session=session,identifier=drv)

    sns.violinplot(df,x='Driver',y='LapTime',hue='Stint_Compound',palette=[color]*df.Stint.nunique(),
                inner=None,linecolor='w',legend=False)
    sns.stripplot(df,x='Driver',y='LapTime',hue='Stint_Compound',palette=comp_palette
                ,jitter=0.08,linewidth=1,size=15,edgecolor='k',dodge=True,legend=False)

# Collect laps
mean_laps = (
    session.laps.pick_drivers(drivers[:5])
    .pick_quicklaps(1.05)
    .copy()
)
mean_laps['LapTime'] = mean_laps['LapTime'].dt.total_seconds()

# Mean per driver+stint
driver_stint_means = mean_laps.groupby(["Driver", "Stint"])["LapTime"].mean().reset_index()

# Pivot: rows=Driver, cols=Stint
pivot_means = driver_stint_means.pivot(index="Driver", columns="Stint", values="LapTime")
pivot_means = pivot_means.loc[['VER','NOR','PIA','LEC','RUS']]

# Plot one line per stint across drivers
for stint in pivot_means.columns[:1]:
    ax.plot(
        [-0.2,0.8,1.8,2.8,3.8],            # x = driver index
        pivot_means[stint].values,          # y = mean laptimes for that stint
        marker="o", linestyle="--", linewidth=2, markersize=10,
        label=f"Stint {stint} Mean Lap Time"
    )

# Plot one line per stint across drivers
for stint in pivot_means.columns[1:]:
    ax.plot(
        [0.2,1.2,2.2,3.2,4.2],            # x = driver index
        pivot_means[stint].values,          # y = mean laptimes for that stint
        marker="o", linestyle="--", linewidth=2, markersize=10,
        label=f"Stint {stint} Mean Lap Time"
    )

ax.legend()
ax.set_ylim(80,86)
ax.set_ylabel("Lap Time (sec)")
ax.set_xlabel("")
ax.set_xticks([])
ax.set_title('Race Pace (lower laptime is better)',pad=30)

ax.grid(visible=False)

In [ ]:
save_fig(fig=fig, name='race_pace',loc='Reel18')